The binomial parameter analysis here is inspired by the following manuscripts:

* Costa et al 2013. https://www.frontiersin.org/articles/10.3389/fncom.2013.00075/full
* Malinow and Tsien 1990. https://www.nature.com/articles/346177a0
* Tarczy-hornoch et al 1999. https://academic.oup.com/cercor/article/9/8/833/346715
* Bremaud et al 2007. https://www.pnas.org/content/104/35/14134.long

In the images below I have have looked at two synapse types, pv to pv, and tlx3 to sst (look for tlx3 to sst are at the bottom).  I have not made summary plots yet.  Each connection has four plots associated with it.  All plots are for deconvolved amplitudes.  It starts with a plot of the amplitudes of the entire experiment.  Then I plot the 50 Htz trains, the mean vs coefficient of variation for each pulse of the 50 htz trains, and histograms of the amplitudes of first pulses from every train (not just 50 htz).


In [ ]:
import pandas as pd 
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
font={'size':14}
matplotlib.rc('font', **font)
#need to add in qc and mean
# still need to add pre synaptic spike number qc. ie.e does more than one presynaptic spike happen?
# what if a postsynaptic pulse happens and it does not pass qc 

In [ ]:
def get_data_from_csv(filename):
    """Get data from a specified csv file.  Note the data arrays were written as strings with 
    funky things in them by pandas.to_csv.  Here they are pulled out and put in a dictionary as
    numpy arrays.
    Input
    -----
    filename: string
        file name or path to file
    Returns
    -------
    d: dictionary
        contains data for connections
        """
    
    data = pd.read_csv(filename)
    d={}
    d['amps'] = []
    d['spike_times_relative_to_expt'] = []
    d['post_syn_rec_id'] = []
    d['expt_start_time'] = []
    d['post_id'] = data['post_id']
    d['pre_id'] = data['pre_id']
    d['post_cre'] = data['post_cre']
    d['pre_cre'] = data['pre_cre']
    d['uid'] = data['uid']
    d['stim_freq'] = []
    d['pulse_num_within_train'] = []
#    d['qc_mask'] = []
    
    #convert weird string from cvs to arrays
    for ii in range(len(data)):
        d['amps'].append(np.fromstring(data['deconvolved_amps'][ii].replace('\n', '').replace('[', '').replace(']', ''), dtype=float, sep=' '))
        d['spike_times_relative_to_expt'].append(np.fromstring(data['spike_times_relative_to_experiment'][ii].replace('\n', '').replace('[', '').replace(']', ''), dtype=float, sep=' '))
        d['expt_start_time'].append(d['spike_times_relative_to_expt'][-1][0])
        d['post_syn_rec_id'].append(np.fromstring(data['post_syn_rec_id'][ii].replace('\n', '').replace('[', '').replace(']', ''), dtype=int, sep=' '))
        d['pulse_num_within_train'].append(np.fromstring(data['pulse_num_within_train'][ii].replace('\n', '').replace('[', '').replace(']', ''), dtype=int, sep=' '))
        #print(data['stim_freq'][ii])
        d['stim_freq'].append(np.fromstring(data['stim_freq'][ii].replace('\n', '').replace('[', '').replace(']', ''), dtype=float, sep=' '))
        if (len(d['amps'][-1]) != len(d['spike_times_relative_to_expt'][-1])) or \
            (len(d['amps'][-1]) != len(d['post_syn_rec_id'][-1])) or \
            (len(d['amps'][-1]) != len(d['stim_freq'][-1])) or \
            (len(d['amps'][-1]) != len(d['pulse_num_within_train'][-1])):
            raise Exception('string to array conversion probably went wrong')
    return d

filename='pv_tlx_to_sst.csv'
data=get_data_from_csv(filename)

In [ ]:
def var_std_of_amps_in_df(df):
    """standard deviation of each pulse in train.  Doing
    this calculation by had here because df['amps'].std() is not working."""
    stuff=[]
    for ii, row in df.iterrows(): #/fifty['times_rel_to_train'].mean() 
        stuff.append(row['amps'])
    var=np.array(stuff).var(axis=0)
    std=np.array(stuff).std(axis=0)
    return var, std  

def remove_neg_coupled_data(a,b):
    """
    Remove negative data from arrays.  If a number is negative in a it's pair is removed
    from b and visa versa.
    
    Input
    -----
    a : numpy array
    b : numpy array
    
    a and b should have the same length
    
    Returns
    -------
    a2 : numpy array
    b2 : numpy array
    """
    
    
    if len(a) != len(b):
        raise Exception('input arrays should be the same length')
    a1 = np.extract(a>0, a) #remove negative a data from a
    b1 = np.extract(a>0, b) #remove negative a data indexs from b
    
    a2 = np.extract(b1>0, a1) #remove negative b data index from a
    b2 = np.extract(b1>0, b1) #remove negative b data from b
    
    if len(a2) != len(b2):
        raise Exception('output arrays should be the same length')
    
    return a2, b2

In [ ]:
# Grab 50 Htz data and make plots if desired

show_plot= False
summary_50htz={}
summary_50htz['mean'] = []
summary_50htz['cv'] = []
summary_50htz['uid'] = []
summary_50htz['pre_id'] = []
summary_50htz['post_id'] = []
summary_50htz['pre_cre'] = []
summary_50htz['post_cre'] = []
summary_50htz['std'] = []
summary_50htz['var'] =[]
for ii in range(len(data['uid'])):
    sample=range(0,len(data['amps'][ii]))

    # plot whole experiment
    if show_plot == True:
        plt.figure(figsize=(15,3))
        plt.plot(data['spike_times_relative_to_expt'][ii][sample], data['amps'][ii][sample], '.-', ms=20)
        # for ii, (p,t,a) in enumerate(zip(pulse_ids[sample], spike_times_relative_to_expt[sample], amplitudes[sample]*1e3)):
        #     plt.annotate(str(ii), xy=(t,a), textcoords='data')
        plt.ylabel('Deconvolution Amp')
        plt.xlabel('time (s)')
        plt.title('Deconvolution Amplitude Of Entire Experiment (all freq), \n%.3f, pre: %s %i, post: %s %i' %(data['uid'][ii], data['pre_cre'][ii], data['pre_id'][ii], data['post_cre'][ii], data['post_id'][ii]))

    # create new dictionary for individual connections where trains are differentiated.
    conn_dict={}
    conn_dict['train_num'] = []
    conn_dict['post_rec_id'] = []
    conn_dict['amps'] = []
    conn_dict['times_rel_to_expt'] = []
    conn_dict['times_rel_to_train'] = []
    conn_dict['pulse_num_within_train'] = [] 
    conn_dict['stim_freq'] = []
    

    start_train_index = 0
    end_train_index = 0
    train_num = 1
    recording_id = data['post_syn_rec_id'][ii]
    
    # Walk though every pulse and group them acording to the recording id
    for jj in range(1, len(recording_id)):
        if (recording_id[jj-1] != recording_id[jj]):
            end_train_index = jj-1  
            #make the plot
            sample = range(start_train_index, end_train_index+1) #+1 due to range
            train_stim_freq=np.unique(data['stim_freq'][ii][sample])
            rec_id=np.unique(data['post_syn_rec_id'][ii][sample])

            # sanity checks
            if (len(train_stim_freq) != 1) or (len(rec_id) != 1):
                raise Exception('train should have one freq and one rec_id')
            
            # make data types more convenient
            train_stim_freq = np.int64(train_stim_freq[0])
            rec_id = rec_id[0]
            
            train_times_rel_to_expt = data['spike_times_relative_to_expt'][ii][sample]
            train_times_rel_to_train = data['spike_times_relative_to_expt'][ii][sample]-data['spike_times_relative_to_expt'][ii][start_train_index]
            train_amps = data['amps'][ii][sample]
            pulse_num_within_train = data['pulse_num_within_train'][ii][sample]
#            plt.plot(train_times_rel_to_train, train_amps, '.-', ms=20)
            start_train_index = jj
    
            #TODO: remember to eliminate trains with missing pre synaptic spikes
            if np.all(pulse_num_within_train[0:8] == np.array([1,2,3,4,5,6,7,8])):
                conn_dict['train_num'].append(train_num)
                conn_dict['post_rec_id'].append(rec_id)
                if data['pre_cre'][ii]=='pvalb':
                    conn_dict['amps'].append(-train_amps[0:8]) #you could negate them here
                else:
                    conn_dict['amps'].append(train_amps[0:8]) #you could negate them here

                conn_dict['times_rel_to_expt'].append(train_times_rel_to_expt[0:8])
                conn_dict['times_rel_to_train'].append(train_times_rel_to_train[0:8])
                conn_dict['pulse_num_within_train'].append(pulse_num_within_train[0:8])
                conn_dict['stim_freq'].append(train_stim_freq)
            
            train_num += 1
    df=pd.DataFrame(conn_dict)

    # create 50 htz df
    fifty=df[['times_rel_to_train', 'amps']][df['stim_freq']==50]
    if len(fifty['amps'])<1:  # if there is not at least 1 train jump out of loop
        print('No Fifty Htz trains: %.3f, pre: %i, post: %i' % \
              (data['uid'][ii], data['pre_id'][ii], data['post_id'][ii]))
        continue

    # plot individual fifty hertz trains        
    if show_plot == True:
        plt.figure(figsize=(15,5))
        for kk, row in fifty.iterrows():
            # plot each train
            plt.plot(range(1,9), row['amps'], '.-', ms=20)
        plt.plot(range(1,9), fifty['amps'].mean(), '.-k', lw=5, ms=30)
        plt.xlabel('Pulse #')
        plt.ylabel('Deconvoled Amplitude')
        plt.title('50 Hz Trains')
    
    # get amp stats for 50 htz trains
    mean = fifty['amps'].mean()
    var, std = var_std_of_amps_in_df(fifty)
    
    #remove_neg_coupled_data could be improved to handle multiple arrays so dont have to do hack below.
    mean1, std1 = remove_neg_coupled_data(mean, std) 
    mean1, var1 = remove_neg_coupled_data(mean, var) 
    mean = mean1
    std = std1
    var = var1

    cv = std/mean
    
    # populate summary dictionary
    summary_50htz['cv'].append(cv)
    summary_50htz['std'].append(std)
    summary_50htz['mean'].append(mean)
    summary_50htz['var'].append(var)
    summary_50htz['uid'].append(data['uid'][ii])
    summary_50htz['pre_id'].append(data['pre_id'][ii])
    summary_50htz['post_id'].append(data['post_id'][ii])
    summary_50htz['pre_cre'].append(data['pre_cre'][ii])
    summary_50htz['post_cre'].append(data['post_cre'][ii])
    
    # plot coefficient of variation at each spike index for 50 Htz trains
    if show_plot == True:
        plt.figure(figsize=(15,5))
        plt.plot(range(1,9), cv, ms=20)
        plt.xlabel('Pulse #')
        plt.ylabel(' CV of Deconvoled Amp')
        plt.title('50 Hz Trains')

        

    
    
    # plot amp mean and coefficient of variation for 50 htz trains
    if show_plot == True:
        plt.figure(figsize=(15,6))
        plt.subplot(1,2,1)    
        plt.plot(mean, cv, ms=20)
        for mm, (a, c) in enumerate(zip(mean, cv)):
            plt.annotate(str(mm+1), (a,c))
        plt.xlabel('Mean deconvolved amplitude')
        plt.ylabel('CV deconvolved amplitude')
        plt.title('50 Htz trains')

    
    # first pulse histograms
    if show_plot == True:
        plt.subplot(1,2,2)    
        fp=[]
        for row in df['amps']:
            fp.append(row[0])
        plt.hist(fp)
        plt.xlabel('Deconvolved amplitude')
        plt.ylabel('Number')
        plt.title('First Pulse (all frequencies)')
    
        plt.show()

summary_df=pd.DataFrame(summary_50htz)

In [ ]:
summary_df['mean'][0]


In [ ]:
# make 1D long arrays for fitting by concatenating idividual trains (note that all pulses are included)

# initiate data structures
tlx3_to_sst = {'mean':{'list':[], 'array': []},
               'var':{'list':[], 'array': []},
               'cv2':{'list':[], 'array': []},
               'cv_log':{'list':[], 'array': []}}

pv_to_pv = {'mean':{'list':[], 'array': []},
            'var':{'list':[], 'array': []},
            'cv2':{'list':[], 'array': []},
            'cv_log':{'list':[], 'array': []}}

# Put dataframe in a list of arrays. Probably a better way to do this in pandas but not sure how
for ii, row in summary_df.iterrows():
    if row['pre_cre']=='tlx3':
        tlx3_to_sst['mean']['list'].append(row['mean'])
        tlx3_to_sst['cv2']['list'].append(row['cv']**2)
        tlx3_to_sst['cv_log']['list'].append(np.log(row['cv']))
        tlx3_to_sst['var']['list'].append(row['var'])

    if row['pre_cre']=='pvalb':
        pv_to_pv['mean']['list'].append(row['mean']) 
        pv_to_pv['cv2']['list'].append(row['cv']**2)
        pv_to_pv['cv_log']['list'].append(np.log(row['cv']))
        pv_to_pv['var']['list'].append(row['var'])

# Do the transformation into 1d from 2d array for analysis on using all pulses
tlx3_to_sst['mean']['array'] = np.concatenate(tlx3_to_sst['mean']['list'])
tlx3_to_sst['cv2']['array'] = np.concatenate(tlx3_to_sst['cv2']['list'])
tlx3_to_sst['cv_log']['array'] = np.concatenate(tlx3_to_sst['cv_log']['list'])
tlx3_to_sst['var']['array'] = np.concatenate(tlx3_to_sst['var']['list'])


pv_to_pv['mean']['array']  = np.concatenate(pv_to_pv['mean']['list'])
pv_to_pv['cv2']['array'] = np.concatenate(pv_to_pv['cv2']['list'])
pv_to_pv['cv_log']['array'] = np.concatenate(pv_to_pv['cv_log']['list'])
pv_to_pv['var']['array'] = np.concatenate(pv_to_pv['var']['list'])


# make some plots
% matplotlib inline
# plt.figure(figsize=(10, 10))
# plt.plot(np.log(pv_to_pv['mean']['array']), np.log(pv_to_pv['cv2']['array']), '.b', ms=20, label='pv to pv')
# plt.plot(np.log(tlx3_to_sst['mean']['array']), np.log(tlx3_to_sst['cv2']['array']), '.r', ms=20, label='tlx3 to sst')
# plt.legend()

a=plt.figure(figsize=(16, 6))
plt.subplot(1,2,1)
plt.plot(pv_to_pv['mean']['array'], pv_to_pv['var']['array'], '.b', ms=20, label='pv to pv')
plt.plot(tlx3_to_sst['mean']['array'], tlx3_to_sst['var']['array'], '.r', ms=20, label='tlx3 to sst')
plt.ylabel('variance of amplitude (deconvolved)')
plt.xlabel('mean amplitute (deconvolved)')
plt.legend()

plt.subplot(1,2,2)
plt.plot(np.log(pv_to_pv['mean']['array']), np.log(pv_to_pv['var']['array']), '.b', ms=20, label='pv to pv')
plt.plot(np.log(tlx3_to_sst['mean']['array']), np.log(tlx3_to_sst['var']['array']), '.r', ms=20, label='tlx3 to sst')
plt.ylabel('log variance of amplitude (deconvolved)')
plt.xlabel('log mean amplitute (deconvolved)')

a.suptitle('Pulses 1:8 included (each point represents data for one pulse: i.e. each connection will have 8 points)', fontsize='16')

In [ ]:
"""Curve fitting with forced on non forced parameters to get to look at tradeoffs in p, n and q."""

# Different potential fits
from scipy.optimize import curve_fit
def fit_func(m, n, q):
    return q/m - 1/n

def f2(m, n, q):
    return 0.5* np.log(q/m - 1/n)

def var_mean_fit_n_q(m, n, q):
    return q*m - (m**2/n)

def var_mean_fit_n_q_linear(m, n, q):
    return q - (m/n)
    
%matplotlib inline
font={'size':22}
matplotlib.rc('font', **font)  

# #---------v vs m non linear space----------------
# #unconstrained fit
#
# popt, pcov = curve_fit(var_mean_fit_n_q, tlx3_to_sst['mean']['array'], tlx3_to_sst['var']['array'])# , bounds=([1, 0], [10., np.inf] ))
# plt.figure(figsize=(10, 10))
# plt.plot(tlx3_to_sst['mean']['array'], tlx3_to_sst['var']['array'], '.b', ms = 10, label='data')   
# plt.plot(tlx3_to_sst['mean']['array'], var_mean_fit_n_q(tlx3_to_sst['mean']['array'], *popt), 'g.', ms=10, label='unconstrained, fit: n=%f, q=%f' % tuple(popt))

# #contstrain fit from n=1 to 10
# colors=['r', 'k', 'y', 'c', 'm']
# for ii, n in enumerate(range(1,6)):
#     popt, pcov = curve_fit(var_mean_fit_n_q, tlx3_to_sst['mean']['array'], tlx3_to_sst['var']['array'],  bounds=([n-.000001, 0], [n+.000001, np.inf] ))
#     plt.plot(tlx3_to_sst['mean']['array'], var_mean_fit_n_q(tlx3_to_sst['mean']['array'], *popt), '.', color=colors[ii],ms=10, label='fit: n=%.0f, q=%f' % tuple(popt))

# plt.legend()
# plt.xlabel('deconvolved amp mean')
# plt.ylabel('deconvolved amp variance ')

# # Do this again in linear space
# plt.figure(figsize=(10, 10))

#----------------------------------------------------------
#------ these are full blown plots ------------------------
#----------------------------------------------------------

#----------tlx-----------------
plt.figure(figsize=(15,12))
x = tlx3_to_sst['mean']['array']
y = tlx3_to_sst['var']['array']/tlx3_to_sst['mean']['array']

#sort values for plotting
arr1inds = x.argsort()
x = x[arr1inds[::-1]]
y = y[arr1inds[::-1]]

#unconstrained fit
popt, pcov = curve_fit(var_mean_fit_n_q_linear, x, y)# , bounds=([1, 0], [10., np.inf] ))
p = x.mean()/(popt[0]*popt[1])
plt.plot(x, y, '.b', ms = 14, label='data, tlx3 to sst')   
plt.plot(x, var_mean_fit_n_q_linear(x, *popt), 'g', ms=10, lw=3,
         label='unconstrained, fit: n=%f, q=%f, p=%f' % (popt[0], popt[1], p))

#contstrain fit from n=1 to 10
colors=['r', 'k', 'y', 'c', 'm']
for ii, n in enumerate(range(1,6)):
    popt, pcov = curve_fit(var_mean_fit_n_q_linear, x, y,  bounds=([n-.000001, 0], [n+.000001, np.inf] ))
    p = x.mean()/(popt[0]*popt[1])
    plt.plot(x, var_mean_fit_n_q_linear(x, *popt), lw=3, color=colors[ii],ms=10, 
             label=' fit: n=%.0f, q=%f, p=%f' % (popt[0], popt[1], p))

    
#----------pv-----------------
y = pv_to_pv['var']['array']/pv_to_pv['mean']['array']
x = pv_to_pv['mean']['array']
#sort values for plotting
arr1inds = x.argsort()
x = x[arr1inds[::-1]]
y = y[arr1inds[::-1]]

popt, pcov = curve_fit(var_mean_fit_n_q_linear, x, y)# , bounds=([1, 0], [10., np.inf] ))
p = x.mean()/(popt[0]*popt[1])
plt.plot(x, y, '.r', ms = 14, label='data: pv to pv')   
plt.plot(x, var_mean_fit_n_q_linear(x, *popt), 'g-', ms=10, lw=3,
         label='unconstrained, fit: n=%f, q=%f, p=%f' % (popt[0], popt[1], p))

#contstrain fit from n=1 to 10
colors=['r', 'k', 'y', 'c', 'm']
for ii, n in enumerate(range(2,20)):
    popt, pcov = curve_fit(var_mean_fit_n_q_linear, x, y,  bounds=([n-.000001, 0], [n+.000001, np.inf]))
    p = x.mean()/(popt[0]*popt[1])
    plt.plot(x, var_mean_fit_n_q_linear(x, *popt), ms=10, lw=3,
             label=' fit: n=%.0f, q=%f, p=%f' % (popt[0], popt[1], p))

plt.ylim([0, .03])
plt.legend(fontsize=16)
plt.xlabel('deconvolved amp mean')
plt.ylabel('deconvolved amp variance/mean ')


In [ ]:
#----------------------------------------------------------
#------ summary plot for Tim's Columbia talk --------------
#----------------------------------------------------------

%matplotlib inline
font={'size':22}
matplotlib.rc('font', **font)  

#----------tlx-----------------
plt.figure(figsize=(15,12))
x = tlx3_to_sst['mean']['array']
y = tlx3_to_sst['var']['array']/tlx3_to_sst['mean']['array']

#sort values for plotting
arr1inds = x.argsort()
x = x[arr1inds[::-1]]
y = y[arr1inds[::-1]]

#unconstrained fit
popt, pcov = curve_fit(var_mean_fit_n_q_linear, x, y)# , bounds=([1, 0], [10., np.inf] ))
p = x.mean()/(popt[0]*popt[1])
plt.plot(x, y, '.b', ms = 20, label='tlx3 to sst')   
plt.plot(x, var_mean_fit_n_q_linear(x, *popt), 'b', ms=10, lw=3)

#contstrain fit from n=1 to 10
colors=['r', 'k', 'y', 'c', 'm']
for ii, n in enumerate(range(1,6)):
    popt, pcov = curve_fit(var_mean_fit_n_q_linear, x, y,  bounds=([n-.000001, 0], [n+.000001, np.inf] ))
    p = x.mean()/(popt[0]*popt[1])
#     plt.plot(x, var_mean_fit_n_q_linear(x, *popt), lw=3, color=colors[ii],ms=10, 
#              label=' fit: n=%.0f, q=%f, p=%f' % (popt[0], popt[1], p))

    
#----------pv-----------------
y = pv_to_pv['var']['array']/pv_to_pv['mean']['array']
x = pv_to_pv['mean']['array']
#sort values for plotting
arr1inds = x.argsort()
x = x[arr1inds[::-1]]
y = y[arr1inds[::-1]]

popt, pcov = curve_fit(var_mean_fit_n_q_linear, x, y)# , bounds=([1, 0], [10., np.inf] ))
p = x.mean()/(popt[0]*popt[1])
plt.plot(x, y, '.r', ms = 20, label='pv to pv')   
plt.plot(x, var_mean_fit_n_q_linear(x, *popt), 'r-', ms=10, lw=3)

#contstrain fit from n=1 to 10
colors=['r', 'k', 'y', 'c', 'm']
for ii, n in enumerate(range(1,20)):
    popt, pcov = curve_fit(var_mean_fit_n_q_linear, x, y,  bounds=([n-.000001, 0], [n+.000001, np.inf]))
    p = x.mean()/(popt[0]*popt[1])
#     plt.plot(x, var_mean_fit_n_q_linear(x, *popt), ms=10, lw=3,
#              label=' fit: n=%.0f, q=%f, p=%f' % (popt[0], popt[1], p))
plt.ylim([0, .03])
plt.legend()
plt.xlabel('mean')
plt.ylabel('variance/mean')
plt.title('Amplitude of 50 Hz Trains')
